In [1]:
# importing libraries
!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import csv
import time
import logging
import os
import re
import json
import random
from urllib.parse import urlparse, parse_qs
from tqdm import tqdm
import sys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 37.1 MB/s eta 0:00:00


In [12]:
# Configure logging with timestamps
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('avito_property_scraper.log'),
        logging.StreamHandler()
    ]
)

class AvitoPropertyScraper:
    def __init__(self, links_file='data/avito_links.csv', output_file='data/avito_properties.csv'):
        self.links_file = links_file
        self.output_file = output_file

        # Setup Selenium options
        chrome_options = Options()
        chrome_options.add_argument("--headless=new")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--lang=fr")
        chrome_options.add_argument("--user-agent=Mozilla/5.0")

        self.driver = webdriver.Chrome(options=chrome_options)

        # Columns to extract
        self.columns = [
            'id', 'url', 'title', 'price', 'location', 'type',
            'area', 'rooms', 'bedrooms', 'bathrooms', 'description',
            'property_state', 'jardin', 'piscine', 'cuisine_equiped',
            'terrasse', 'garage', 'ascenseur', 'quartier', 'status'
        ]

        # Initialize output file if it doesn't exist
        self._initialize_output_file()

        # Track processed URLs to avoid duplicates
        self.processed_urls = self._get_processed_urls()

    def _initialize_output_file(self):
        """Create output file with headers if it doesn't exist"""
        os.makedirs(os.path.dirname(self.output_file), exist_ok=True)
        if not os.path.exists(self.output_file):
            with open(self.output_file, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(self.columns)

    def _get_processed_urls(self):
        """Get list of already processed URLs from the output file"""
        processed = set()
        if os.path.exists(self.output_file):
            try:
                with open(self.output_file, 'r', encoding='utf-8') as f:
                    reader = csv.reader(f)
                    next(reader)  # Skip header
                    for row in reader:
                        if len(row) > 1:  # Ensure row has data
                            processed.add(row[1])  # URL is at index 1
            except Exception as e:
                logging.error(f"Error reading processed URLs: {e}")
        return processed

    def read_links(self):
        """Read URLs from the links file"""
        links = []
        with open(self.links_file, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            for row in reader:
                if row and row[0] and row[0] not in self.processed_urls:
                    links.append(row[0])
        return links

    def extract_id_from_url(self, url):
        """Extract property ID from URL"""
        try:
            path = urlparse(url).path
            # Split path and get the last part which might be the ID
            id_part = path.strip('/').split('/')[-1]
            # If ID is numeric or alphanumeric, return it
            if re.match(r'^[a-zA-Z0-9_-]+$', id_part):
                return id_part
        except:
            pass
        return None

    def extract_text_by_selector(self, selector, method=By.CSS_SELECTOR, default=None):
        """Extract text from an element using CSS selector or XPath"""
        try:
            element = self.driver.find_element(method, selector)
            return element.text.strip()
        except NoSuchElementException:
            return default
        except Exception as e:
            logging.error(f"Error extracting text with selector {selector}: {e}")
            return default

    def extract_title(self):
        """Extract property title"""
        selectors = [
            "h1",  # Generic h1 (usually main title)
            "h1[class*='sc-']",  # h1 with any sc- class
            "[class*='title']"  # Any element with title in class name
        ]

        for selector in selectors:
            title = self.extract_text_by_selector(selector)
            if title:
                return title
        return None

    def extract_price(self):
        """Extract property price"""
        selectors = [
            "p[font-weight='bold']",  # Bold paragraph (often price)
            "[class*='price']",  # Any class containing 'price'
            "p[class*='sc-'][font-weight='bold']"  # Classes with sc- pattern and bold weight
        ]

        for selector in selectors:
            price_text = self.extract_text_by_selector(selector)
            if price_text:
                # Extract numeric value from price text (e.g., "4 400 000 DH" -> 4400000)
                price_numeric = re.sub(r'[^\d]', '', price_text)
                if price_numeric:
                    return price_numeric
        return None

    def extract_location(self):
        """Extract property location"""
        selectors = [
            "span[style*='text-wrap:nowrap']",  # Spans with nowrap style
            "[class*='location']",  # Any class containing 'location'
            "span[class*='sc-'][style*='nowrap']"  # Spans with sc- classes and nowrap style
        ]

        for selector in selectors:
            location = self.extract_text_by_selector(selector)
            if location:
                return location
        return None

    def extract_type(self):
        """Extract property type"""
        selectors = [
            "span:contains('Appartements')",
            "span:contains('Villas')",
            "span:contains('Maisons')",
            "span:contains('Terrains')",
            "[class*='category']",
            "span[class*='sc-']"  # Generic span with sc- class pattern
        ]

        # Since :contains is jQuery, we'll need to adapt for XPath
        for selector in selectors:
            if ":contains(" in selector:
                text = selector.split(":contains('")[1].split("')")[0]
                xpath = f"//span[contains(text(), '{text}')]"
                result = self.extract_text_by_selector(xpath, By.XPATH)
            else:
                result = self.extract_text_by_selector(selector)

            if result:
                return result
        return None

    def extract_area(self):
        """Extract property area in square meters"""
        selectors = [
            "//span[@class='sc-1x0vz2r-0 fjZBup']/following-sibling::div[contains(@class, 'sc-19cngu6-2 kuofIS')]",
            "//div[contains(text(), 'm²')]",
            "//span[contains(text(), 'm²')]"
        ]

        for selector in selectors:
            area_text = self.extract_text_by_selector(selector, By.XPATH)
            if area_text:
                # Extract numeric value and unit
                match = re.search(r'(\d+)\s*m²', area_text)
                if match:
                    return match.group(1)  # Return just the number

    def extract_description(self):
        """Extract property description"""
        selectors = [
            "div[color='midnight_lighter']",  # Based on your screenshot
            "div[class*='description']",  # Any class containing 'description'
            "//h2[contains(text(), 'Description')]/following-sibling::div",  # Div after Description heading
            "div[class*='sc-'][color]"  # Div with sc- class and color attribute
        ]

        for selector in selectors:
            if selector.startswith("//"):
                description = self.extract_text_by_selector(selector, By.XPATH)
            else:
                description = self.extract_text_by_selector(selector)

            if description:
                return description
        return None

    def extract_rooms_info(self):
        """Extract rooms, bedrooms and bathrooms information"""
        rooms = None
        bedrooms = None
        bathrooms = None

        # Try to find specific elements for rooms
        try:
            # Check for icon elements with alt texts
            room_elements = self.driver.find_elements(By.XPATH, "//img[contains(@alt, 'Chambre') or contains(@alt, 'room') or contains(@alt, 'bedroom')]")
            bath_elements = self.driver.find_elements(By.XPATH, "//img[contains(@alt, 'Salle de bain') or contains(@alt, 'bathroom')]")

            # Look for numbers next to these elements
            for elem in room_elements:
                parent = elem.find_element(By.XPATH, "./..")
                number_text = parent.text.strip()
                if number_text and number_text.isdigit():
                    bedrooms = number_text
                    break

            for elem in bath_elements:
                parent = elem.find_element(By.XPATH, "./..")
                number_text = parent.text.strip()
                if number_text and number_text.isdigit():
                    bathrooms = number_text
                    break

        except Exception as e:
            logging.error(f"Error extracting rooms info: {e}")

        # If not found, try to extract from description
        if not bedrooms or not bathrooms:
            description = self.extract_description() or ""

            # Look for bedroom patterns
            bedroom_match = re.search(r'(\d+)\s*chambre', description, re.IGNORECASE)
            if bedroom_match:
                bedrooms = bedroom_match.group(1)

            # Look for bathroom patterns
            bathroom_match = re.search(r'(\d+)\s*salle de bain', description, re.IGNORECASE)
            if bathroom_match:
                bathrooms = bathroom_match.group(1)

        # Calculate total rooms if possible
        if bedrooms:
            # Try to find living rooms (salons)
            salon_match = re.search(r'(\d+)\s*salon', self.extract_description() or "", re.IGNORECASE)
            if salon_match:
                salons = salon_match.group(1)
                # Total rooms = bedrooms + salons
                rooms = str(int(bedrooms) + int(salons))
            else:
                # Assume at least one living room
                rooms = str(int(bedrooms) + 1)

        return rooms, bedrooms, bathrooms

    def check_feature_in_description(self, keywords):
        """Check if any keywords are present in description"""
        description = self.extract_description() or ""
        description = description.lower()

        for keyword in keywords:
            if keyword.lower() in description:
                return True
        return False

    def extract_amenities(self):
        """Extract amenities from description"""
        description = self.extract_description() or ""
        description = description.lower()

        # Initialize amenities with False
        amenities = {
            'jardin': False,
            'piscine': False,
            'cuisine_equiped': False,
            'terrasse': False,
            'garage': False,
            'ascenseur': False
        }

        # Check for each amenity in description
        if any(word in description for word in ['jardin', 'garden']):
            amenities['jardin'] = True

        if any(word in description for word in ['piscine', 'pool']):
            amenities['piscine'] = True

        if any(phrase in description for phrase in ['cuisine équipée', 'cuisine equipee', 'équipée']):
            amenities['cuisine_equiped'] = True

        if any(word in description for word in ['terrasse', 'terrace']):
            amenities['terrasse'] = True

        if any(word in description for word in ['garage', 'parking']):
            amenities['garage'] = True

        if any(word in description for word in ['ascenseur', 'elevator']):
            amenities['ascenseur'] = True

        return amenities

    def extract_quartier(self):
        """Extract neighborhood (quartier) from location or description"""
        location = self.extract_location() or ""

        # Try to extract neighborhood from location (usually first part before comma)
        if ',' in location:
            quartier = location.split(',')[0].strip()
            return quartier

        # Otherwise, look for common neighborhood names in description
        description = self.extract_description() or ""
        neighborhoods = [
            'Hay Riad', 'Agdal', 'Souissi', 'Hassan', 'Les Orangers', 'Médina',
            'Harhoura', 'Temara', 'Sale', 'Kenitra', 'Ain Atiq'
        ]

        for neighborhood in neighborhoods:
            if neighborhood.lower() in description.lower():
                return neighborhood

        return None

    def extract_status(self):
        """Extract property status (for sale, for rent, etc.)"""
        title = self.extract_title() or ""

        # Check for common status indicators
        if any(phrase in title.lower() for phrase in ['à vendre', 'a vendre']):
            return 'à vendre'
        elif any(phrase in title.lower() for phrase in ['à louer', 'a louer']):
            return 'à louer'

        # Try to find status in type field
        type_text = self.extract_type() or ""
        if 'vendre' in type_text.lower():
            return 'à vendre'
        elif 'louer' in type_text.lower():
            return 'à louer'

        return None

    def extract_property_state(self):
        """Extract property state (new, renovated, etc.)"""
        selectors = [
            "//div[contains(@class, 'sc-19cngu6-2 kuofIS')]//span[@class='sc-1x0vz2r-0 fjZBup' and contains(text(), 'Âge du bien')]/following-sibling::span",
            "//div[contains(@class, 'sc-19cngu6-2 kuofIS')]//span[@class='sc-1x0vz2r-0 bXFCIH' and contains(text(), 'Âge du bien')]/following-sibling::span"
        ]

        for selector in selectors:
            state_text = self.extract_text_by_selector(selector, By.XPATH)
            if state_text:
                state_text = state_text.strip().lower()

                # Assuming we want to extract the state directly as found (such as "neuf", "rénové", etc.)
                return state_text

        return None

    def scrape_property(self, url):
        """Scrape a single property listing"""
        property_data = {col: None for col in self.columns}
        property_data['url'] = url

        try:
            self.driver.get(url)
            time.sleep(2)  # Wait for page to load

            # Extract all data
            property_data['id'] = self.extract_id_from_url(url)
            property_data['title'] = self.extract_title()
            property_data['price'] = self.extract_price()
            property_data['location'] = self.extract_location()
            property_data['type'] = self.extract_type()
            property_data['area'] = self.extract_area()

            # Extract rooms info
            rooms, bedrooms, bathrooms = self.extract_rooms_info()
            property_data['rooms'] = rooms
            property_data['bedrooms'] = bedrooms
            property_data['bathrooms'] = bathrooms

            property_data['description'] = self.extract_description()
            property_data['property_state'] = self.extract_property_state()

            # Extract amenities
            amenities = self.extract_amenities()
            for key, value in amenities.items():
                property_data[key] = str(value).lower()

            property_data['quartier'] = self.extract_quartier()
            property_data['status'] = self.extract_status()

            return property_data

        except Exception as e:
            logging.error(f"Error scraping property {url}: {e}")
            return property_data

    def save_property(self, property_data):
        """Save property data to CSV file (append mode)"""
        with open(self.output_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            row = [property_data.get(col, None) for col in self.columns]
            writer.writerow(row)
            # Flush to ensure data is written immediately
            f.flush()
            os.fsync(f.fileno())

    def print_progress(self, current, total, property_data):
        """Print detailed progress information"""
        # Clear the current line
        sys.stdout.write('\r' + ' ' * 100 + '\r')

        # Print progress
        progress = f"[{current}/{total}] {(current/total)*100:.1f}% - "

        # Add property details
        title = property_data.get('title', 'Unknown')[:30]
        price = property_data.get('price', 'Unknown')
        location = property_data.get('location', 'Unknown')[:20]

        details = f"{title}... | {price} DH | {location}"
        sys.stdout.write(f"{progress}{details}")
        sys.stdout.flush()

    def scrape_all_properties(self):
        """Scrape all properties from the links file"""
        links = self.read_links()
        total = len(links)

        if total == 0:
           logging.info("No new links to process. All URLs have been already scraped.")
           return

        logging.info(f"Starting to scrape {total} properties")
        print(f"\nStarting to scrape {total} new properties...\n")

        try:
            for i, url in enumerate(links):
                if url in self.processed_urls:
                   continue

                print(f"\n[{i+1}/{total}] Processing: {url}")

                try:
                    property_data = self.scrape_property(url)
                    self.print_progress(i+1, total, property_data)
                    self.save_property(property_data)
                    self.processed_urls.add(url)
                    logging.info(f"Saved property: {property_data.get('title', 'Unknown')}")
                    time.sleep(0.5)

                except Exception as e:
                      logging.error(f"Error scraping {url}: {e}")
                      continue

        except KeyboardInterrupt:
              logging.info("Scraping interrupted by user")
              print("\nScraping interrupted. Data saved up to this point.")

        finally:
               self.driver.quit()
               logging.info(f"Finished scraping. Processed {len(self.processed_urls)} properties total.")
               print(f"\nFinished scraping. Processed {len(self.processed_urls)} properties total.")


if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs('data', exist_ok=True)

    # Check if links file exists
    links_file = 'data/avito_links.csv'
    if not os.path.exists(links_file):
        print(f"Links file not found: {links_file}")
        print("Please run the link collector script first.")
        sys.exit(1)

    # Start scraping
    scraper = AvitoPropertyScraper()
    scraper.scrape_all_properties()


Starting to scrape 1210 new properties...


[1/1210] Processing: https://www.avito.ma/fr/agdal/appartements/apparemment_ensoleill%C3%A9__agdal_54921890.htm


ERROR:root:Error scraping https://www.avito.ma/fr/agdal/appartements/apparemment_ensoleill%C3%A9__agdal_54921890.htm: 'NoneType' object is not subscriptable


                                                                                                    
[2/1210] Processing: https://www.avito.ma/fr/agdal/villas_et_riads/villa_usage_commerciale_%C3%A0_vendre__55014303.htm


ERROR:root:Error scraping https://www.avito.ma/fr/agdal/villas_et_riads/villa_usage_commerciale_%C3%A0_vendre__55014303.htm: 'NoneType' object is not subscriptable


                                                                                                    
[3/1210] Processing: https://www.avito.ma/fr/centre/appartements/Appartement_de_prestige_r%C3%A9sidence_balzac_43696790.htm


...

                                                                                                    
[818/1210] Processing: https://www.avito.ma/fr/souissi/villas_et_riads/villa_r%C3%A9nov%C3%A9e_%C3%A0_OLM_souissi__52574934.htm
[818/1210] 67.6% - villa rénovée à OLM souissi... | 12000000 DH | Souissi, Rabat
[819/1210] Processing: https://www.avito.ma/fr/souissi/villas_et_riads/villa_rabat_souissi_56115736.htm
[819/1210] 67.7% - villa rabat souissi... | 15000000 DH | Souissi, Rabat
[820/1210] Processing: https://www.avito.ma/fr/souissi/villas_et_riads/villa_souissi_megamall_2000m_56218816.htm
[820/1210] 67.8% - villa souissi megamall 2000m... | 14000000 DH | Souissi, Rabat
[821/1210] Processing: https://www.avito.ma/fr/souissi/villas_et_riads/villa_sur_la_vall%C3%A9e_Rabat_onep__54401312.htm
[821/1210] 67.9% - villa sur la vallée Rabat onep... | None DH | Souissi, Rabat
[822/1210] Processing: https://www.avito.ma/fr/souissi/villas_et_riads/villa_sur_la_vall%C3%A9e__52812747.htm
[822/12

ERROR:root:Error scraping https://www.avito.ma/fr/yacoub_el_mansour/appartements/Duplex_%C3%A0_vendre_100_m%C2%B2_%C3%A0_Rabat_enti%C3%A8rement_r%C3%A9nov%C3%A9_56368766.htm: 'NoneType' object is not subscriptable


                                                                                                    
[1073/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/appartements/Joli_Appart_Haut_Standing_de_140_%C3%A0_Resi_El_Manzeh__56279034.htm
[1073/1210] 88.7% - Joli Appart Haut Standing de 1... | 1850000 DH | Yacoub El Mansour, R
[1074/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/appartements/Joli_Appart_de_130m%C2%B2_%C3%A0_R%C3%A9s_Sabah_Cym_Rabat_56296848.htm
[1074/1210] 88.8% - Joli Appart de 130m² à Rés Sab... | 1380000 DH | Yacoub El Mansour, R
[1075/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/appartements/Jolie_appartement__52385771.htm
[1075/1210] 88.8% - Jolie appartement... | 360000 DH | Yacoub El Mansour, R
[1076/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/appartements/RDC_%C3%A0_vendre_carrousel_Projet_Imkan__55006862.htm
[1076/1210] 88.9% - RDC à vendre carrousel Projet ... | 5000000 DH | Yacoub El Mansour, R
[1077/1210

ERROR:root:Error scraping https://www.avito.ma/fr/yacoub_el_mansour/maisons_et_villas/Magnifique_Duplex_Avec_Vue_Mer_Exceptionnelle_56393662.htm: 'NoneType' object is not subscriptable


                                                                                                    
[1135/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/maisons_et_villas/Maison_%C3%A0_3_%C3%A9tages_rue_massira_56272310.htm
[1135/1210] 93.8% - Maison à 3 étages rue massira... | None DH | Yacoub El Mansour, R
[1136/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/maisons_et_villas/Maison_%C3%A0_Mharig_48156081.htm
[1136/1210] 93.9% - Maison à Mharig... | 1900000 DH | Yacoub El Mansour, R
[1137/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/maisons_et_villas/Maison_%C3%A0_Rabat__55264113.htm
[1137/1210] 94.0% - Maison à Rabat... | 1070000 DH | Yacoub El Mansour, R
[1138/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/maisons_et_villas/Maison_%C3%A0_vendre_%C3%A0_Rabat__55404169.htm
[1138/1210] 94.0% - Maison à vendre à Rabat... | 3000000 DH | Yacoub El Mansour, R
[1139/1210] Processing: https://www.avito.ma/fr/yacoub_el_mansour/maisons_et